In [0]:
#partial code borrowed from https://github.com/v1shwa/document-similarity

#need to download google word2vec pretrained model from https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
#need to figure out how to do this. maybe something like wget

#packages to install
#numpy
#gensim
#fasttext

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [0]:
import os
os.chdir('gdrive/My Drive/project_folder/TextSimilarityUsingWord2Vec')
#%cd gdrive/My Drive/project_folder/TextSimilarityUsingWord2Vec


In [41]:
# ! git pull

!git add .
!git commit -m "just changed to colab format"
!git push

[master 38ec3c5] just changed to colab format
 1 file changed, 1 insertion(+), 314 deletions(-)
 rewrite word2vecTest.ipynb (99%)
fatal: could not read Username for 'https://github.com': No such device or address


In [0]:
!git config --global user.email "sanmesh1@gmail.com"
!git config --global user.name "sanmeshkumar udhayakumar"

In [30]:
modelChosen = 'word2vec'

if modelChosen == 'word2vec':
    #load the pretrained word2Vec model
    %cd data/
    !ls
    !wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
    !gunzip GoogleNews-vectors-negative300.bin
    %cd ..
    from gensim.models.keyedvectors import KeyedVectors
    model_path = './data/GoogleNews-vectors-negative300.bin'
    w2v_model = KeyedVectors.load_word2vec_format(model_path, binary=True)
elif modelChosen == 'fasttext':
    #load the pretrained fastText model
    model_path = "data/wiki.en/wiki.en.bin"
#     import fasttext    
#     w2v_model = fasttext.load_model(model_path)    
    from gensim.models.wrappers import FastText
    model = FastText.load_fasttext_format(model_path)


/content/gdrive/My Drive/project_folder/TextSimilarityUsingWord2Vec/data
GoogleNews-vectors-negative300.bin
kaggle_ecommerce_complete_Rank_Crowd_AGG_Descriptions_ALL_ROWS.csv
stopwords_en.txt
test_data.txt
--2020-05-28 05:49:57--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.21.173
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.21.173|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  64.4MB/s    in 24s     

2020-05-28 05:50:22 (64.4 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]

gzip: GoogleNews-vectors-negative300.bin: unknown suffix -- ignored
/content/gdrive/My Drive/project_folder/TextSimilarityUsingWord2Vec


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [29]:
#debuggin code. Leave it commented out

# %cd TextSimilarityUsingWord2Vec/data/
# !ls

# %cd ..

# !gunzip GoogleNews-vectors-negative300.bin

/content/gdrive/My Drive/project_folder/TextSimilarityUsingWord2Vec


In [0]:
#import the model into the document
from DocSim import DocSim
ds = DocSim(w2v_model)

In [0]:
#Create pandas dataframe on dataset
import pandas as pd
#https://data.world/crowdflower/ecommerce-search-relevance
df = pd.read_csv('https://query.data.world/s/iccclhgzbyedtv54p5lresglnun3qz', encoding = "ISO-8859-1")

In [0]:
#create source document and target document we will be comparing scores for
numDataPoints = 6000

sourceTitle = df['product_title'][2587]
source_doc = df['product_description'][2587]

targetTitles = df['product_title'][:numDataPoints].tolist()
target_docs = df['product_description'][:numDataPoints].tolist()

In [0]:
#debugging prints

#print(sourceTitle)
#print(source_doc)
#print(targetTitles[:2])
#print(target_docs[:2])
#print(type(source_doc))
#print(type(target_docs))
#print(type(['delete a invoice', 'how do i remove an invoice', 'purge an invoice']))

In [37]:
# Showing scores for all poducts above a certain threshold
threshold = 0.5
mod_target = target_docs
# Initialise data to lists.  
#data = [{'Title': 'Score', 'For': 'using', 'geeks': 'list'}, 
#        {'Geeks':10, 'For': 20, 'geeks': 30}]  
## Creates DataFrame.  
#df = pd.DataFrame(data) 

finalListOfDicts = []




print("Title of source product = ", sourceTitle)
print("")
print("Description of source product = ", source_doc)
print("")
print("Items with similarity threshold =  ", threshold, " are below:")
for i in range(len(mod_target)):
    if mod_target[i] == mod_target[i]:
        #print(i)
        sim_scores = ds.calculate_similarity(source_doc, mod_target[i])
        if sim_scores != [] and sim_scores[0]['score'] > threshold:
            #print(targetTitles[i])
            #print(sim_scores[0]['score'])
            finalListOfDicts.append({'Title': targetTitles[i], 'score': sim_scores[0]['score']})
            
        
dfFinal = pd.DataFrame(finalListOfDicts)
#dfFinal.sort_values(by=['score'], axis = 1, ascending=False)
dfNew = dfFinal.sort_values(by=['score'], ascending = False)
pd.set_option('display.max_colwidth', -1)
dfNew
#print(sim_scores)

Title of source product =  INSTEN Rechargeable Lithium-ion Battery CR123A for CR123/ CR17345/ DL123/ DL123ABU/ PL123 / BR2/ 3A/ K123LA-1

Description of source product =  This is an INSTEN battery. Save money and protect the environment by using the rechargeable batteries.

Items with similarity threshold =   0.5  are below:


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


,Title,score
356,INSTEN Rechargeable Lithium-ion Battery CR123A for CR123/ CR17345/ DL123/ DL123ABU/ PL123 / BR2/ 3A/ K123LA-1,1.000000
1502,Insten Folio Flip Leather Phone Case for Samsung Galaxy Note 4,0.778602
2673,INSTEN Wii- Dual Charging Station with 2 Rechargeable Batteries,0.760018
1179,Renogy 20 Amp PWM Dual Battery Charge Controller,0.758147
1178,Renogy 10 Amp PWM Dual Battery Charge Controller,0.758147
...,...,...
1006,Marika Flat Rib Waist Casual Pants,0.500561
836,Rashbi Men's Blue Dress Shirt,0.500462
1114,Cuisinart DCC-1100BK Black 12-cup Programmable Coffee Maker,0.500284
848,Fossil Women's ES3225 'Georgia' Crystal Accent Watch,0.500258
